In [1]:
import pandas as pd

In [5]:

df = pd.read_csv("UNIUSDT-trades-2022-12.csv",
                names = ["id","price","qty","quote_qty","time","is_buyer_maker","is_best_match"])

df.time = pd.to_datetime(df.time, unit="ms")
df.set_index("time", inplace = True)
df

,id,price,qty,quote_qty,is_buyer_maker,is_best_match
time,,,,,,
2022-12-01 00:00:01.399,105180671,5.860,42.37,248.28820,False,True
2022-12-01 00:00:07.606,105180672,5.860,14.12,82.74320,False,True
2022-12-01 00:00:08.629,105180673,5.860,17.81,104.36660,False,True
2022-12-01 00:00:09.424,105180674,5.860,3.95,23.14700,False,True
2022-12-01 00:00:12.780,105180675,5.860,5.44,31.87840,False,True
...,...,...,...,...,...,...
2022-12-31 23:59:56.101,105905848,5.166,10.05,51.91830,False,True
2022-12-31 23:59:56.124,105905849,5.166,21.67,111.94722,False,True
2022-12-31 23:59:56.163,105905850,5.166,68.28,352.73448,False,True


In [8]:
df["buy_vol"] = df.qty * (1 - df["is_buyer_maker"])
df["sell_vol"] = df.qty * df["is_buyer_maker"]
df["vol_delta"] = df["buy_vol"] - df["sell_vol"]

df

,id,price,qty,quote_qty,is_buyer_maker,is_best_match,buy_vol,sell_vol,vol_delta
time,,,,,,,,,
2022-12-01 00:00:01.399,105180671,5.860,42.37,248.28820,False,True,42.37,0.0,42.37
2022-12-01 00:00:07.606,105180672,5.860,14.12,82.74320,False,True,14.12,0.0,14.12
2022-12-01 00:00:08.629,105180673,5.860,17.81,104.36660,False,True,17.81,0.0,17.81
2022-12-01 00:00:09.424,105180674,5.860,3.95,23.14700,False,True,3.95,0.0,3.95
2022-12-01 00:00:12.780,105180675,5.860,5.44,31.87840,False,True,5.44,0.0,5.44
...,...,...,...,...,...,...,...,...,...
2022-12-31 23:59:56.101,105905848,5.166,10.05,51.91830,False,True,10.05,0.0,10.05
2022-12-31 23:59:56.124,105905849,5.166,21.67,111.94722,False,True,21.67,0.0,21.67
2022-12-31 23:59:56.163,105905850,5.166,68.28,352.73448,False,True,68.28,0.0,68.28


In [9]:
vol_delta = df["vol_delta"]
vol_delta

time
2022-12-01 00:00:01.399    42.37
2022-12-01 00:00:07.606    14.12
2022-12-01 00:00:08.629    17.81
2022-12-01 00:00:09.424     3.95
2022-12-01 00:00:12.780     5.44
                           ...  
2022-12-31 23:59:56.101    10.05
2022-12-31 23:59:56.124    21.67
2022-12-31 23:59:56.163    68.28
2022-12-31 23:59:56.163     9.00
2022-12-31 23:59:56.251    -9.00
Name: vol_delta, Length: 725182, dtype: float64

In [15]:

cvd = vol_delta.groupby(pd.Grouper(freq="7D")).cumsum()
cvd

time
2022-12-01 00:00:01.399        42.37
2022-12-01 00:00:07.606        56.49
2022-12-01 00:00:08.629        74.30
2022-12-01 00:00:09.424        78.25
2022-12-01 00:00:12.780        83.69
                             ...    
2022-12-31 23:59:56.101   -109692.87
2022-12-31 23:59:56.124   -109671.20
2022-12-31 23:59:56.163   -109602.92
2022-12-31 23:59:56.163   -109593.92
2022-12-31 23:59:56.251   -109602.92
Name: vol_delta, Length: 725182, dtype: float64

In [20]:
cvd_candles = cvd.resample("1H").agg(["max","min","first","last"])
cvd_candles.columns = ["high","low","open","close"]
cvd_candles

,high,low,open,close
time,,,,
2022-12-01 00:00:00,5630.50,-11908.10,42.37,2522.97
2022-12-01 01:00:00,16358.93,-22101.05,2474.06,-19309.01
2022-12-01 02:00:00,-12079.30,-30532.61,-19304.92,-13449.10
2022-12-01 03:00:00,-9268.19,-23888.27,-13418.10,-19546.75
2022-12-01 04:00:00,-18654.51,-29237.21,-19542.50,-20062.29
...,...,...,...,...
2022-12-31 19:00:00,-100753.00,-111809.39,-108159.21,-104387.69
2022-12-31 20:00:00,-102736.94,-104689.99,-104272.20,-103006.15
2022-12-31 21:00:00,-102721.61,-109599.86,-102991.74,-109593.84


In [22]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(
            x = cvd_candles.index,
            open = cvd_candles.open,
            high = cvd_candles.high,
            low = cvd_candles.low,
            close = cvd_candles.close)])

fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()



In [23]:
price_candles = df.price.resample("1H").agg(["max","min","first","last"])
price_candles.columns = ["high","low","open","close"]
price_candles

,high,low,open,close
time,,,,
2022-12-01 00:00:00,5.880,5.790,5.860,5.850
2022-12-01 01:00:00,5.850,5.750,5.840,5.780
2022-12-01 02:00:00,5.800,5.740,5.780,5.800
2022-12-01 03:00:00,5.830,5.780,5.800,5.800
2022-12-01 04:00:00,5.810,5.770,5.810,5.780
...,...,...,...,...
2022-12-31 19:00:00,5.203,5.157,5.161,5.171
2022-12-31 20:00:00,5.202,5.170,5.171,5.192
2022-12-31 21:00:00,5.199,5.166,5.193,5.167


In [26]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig = fig.add_trace(go.Candlestick(
            x = price_candles.index,
            open = price_candles.open,
            high = price_candles.high,
            low = price_candles.low,
            close = price_candles.close),
        row=1, col=1)

fig = fig.add_trace(go.Candlestick(
            x = cvd_candles.index,
            open = cvd_candles.open,
            high = cvd_candles.high,
            low = cvd_candles.low,
            close = cvd_candles.close),
        row=2, col=1)

fig.update_xaxes(rangeslider_visible=False)

fig.show()